### **Installing the Library**

In [ ]:
!pip install streamlit pyngrok --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.0 MB/s eta 0:00:00


### **App.py**

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import time
import numpy as np

st.markdown("""
    <style>
        html, body, [class*="css"] {
            background-color: #141414 !important;
            color: #ffffff !important;
        }

        .stTextInput input {
           background-color: #1f1f1f;
           color: white;
           border: 1px solid #e50914;
           border-radius: 10px;
           padding: 8px;
        }

        .stButton>button {
            background-color: #e50914;
            color: white;
            border: none;
            border-radius: 25px;
            padding: 10px 24px;
            font-weight: bold;
        }

        .stButton>button:hover {
            background-color: #f40612;
            transform: scale(1.05);
            transition: all 0.3s ease;
        }

        .recommendation-card {
            background: #1f1f1f;
            border-radius: 10px;
            padding: 20px;
            margin: 10px 0;
            box-shadow: 0 4px 12px rgba(0,0,0,0.4);
            transition: all 0.3s ease;
        }

        .recommendation-card:hover {
            transform: translateY(-5px);
            box-shadow: 0 8px 16px rgba(229, 9, 20, 0.3);
        }

        .title {
            font-size: 2.8rem;
            text-align: center;
            margin-bottom: 0.5rem;
            color: #e50914;
        }

        .subtitle {
            text-align: center;
            margin-bottom: 2rem;
            font-size: 1.2rem;
            color: #cccccc;
        }

        .more-info-button {
            background-color: #e50914;
            color: white;
            border: none;
            border-radius: 5px;
            padding: 8px 16px;
            font-weight: bold;
            text-decoration: none;
            display: inline-block;
            margin-top: 10px;
        }

        .more-info-button:hover {
            background-color: #f40612;
            transform: scale(1.05);
        }
    </style>
""", unsafe_allow_html=True)

# Load Data
@st.cache_data
def load_data():
    df = pd.read_csv("NetflixDataset.csv", encoding="ISO-8859-1")
    df = df[['Title', 'Genre', 'Tags', 'Summary', 'IMDb Score', 'Image']]
    df.fillna('', inplace=True)
    df['Combined'] = df['Genre'] + ' ' + df['Tags'] + ' ' + df['Summary']
    df['Title_lower'] = df['Title'].str.lower()
    return df

df = load_data()

# Compute Similarity Matrix
@st.cache_data
def compute_similarity(data):
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(data['Combined'])
    similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)
    title_index = pd.Series(data.index, index=data['Title_lower'])
    return similarity, title_index

cosine_sim, title_to_index = compute_similarity(df)

# Recommend Function
def recommend(title, num_results=5):
    title = title.lower()
    if title not in title_to_index:
        return pd.DataFrame()
    idx = title_to_index[title]
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Handle both array and scalar cases
    processed_scores = []
    for i, score in sim_scores:
        if hasattr(score, '__len__'):
            processed_scores.append((i, float(score[0])))
        else:
            processed_scores.append((i, float(score)))

    processed_scores = sorted(processed_scores, key=lambda x: x[1], reverse=True)
    top_indices = [i[0] for i in processed_scores[1:num_results+1]]
    return df.iloc[top_indices]

# App Header
st.markdown('<h1 class="title">🎬 Netflix Movie Recommender</h1>', unsafe_allow_html=True)
st.markdown('<p class="subtitle">🧠 Discover your next binge-worthy obsession 📺</p>', unsafe_allow_html=True)

# Input Section
input_title = st.text_input("🔍 Search for a movie or series:", "", key="search_input")
search_btn = st.button("Find Recommendations", key="search_btn")

# Recommendations
if input_title and search_btn:
    with st.spinner('🔎 Searching the Netflix universe...'):
        time.sleep(1)

    recommendations = recommend(input_title, 5)

    if not recommendations.empty:
        original_title = input_title.title()
        st.success(f"We found {len(recommendations)} perfect matches for **{original_title}**")

        for idx, row in recommendations.iterrows():
            google_query = row['Title'].replace(' ', '+') + "+Netflix"
            google_link = f"https://www.google.com/search?q={google_query}"

            st.markdown(
                f"""
                <div class="recommendation-card">
                    <div style="display: flex; gap: 20px;">
                        <div>
                            <img src="{row['Image'] if row['Image'] else 'https://via.placeholder.com/150x225?text=No+Image'}" width="150" style="border-radius: 5px;">
                        </div>
                        <div style="flex: 1;">
                            <h3 style="color:white;">🎬 {row['Title']}</h3>
                            {"<p style='color:#fff;'>⭐ <strong>IMDb Score:</strong> " + str(row['IMDb Score']) + "/10</p>" if row['IMDb Score'] else ""}
                            <p style="color:#ccc;"><strong>📌 Genre:</strong> {row['Genre']}</p>
                            <p style="color:#aaa;"><strong>📝 Summary:</strong> {row['Summary'][:250]}...</p>
                            <a href="{google_link}" target="_blank" class="more-info-button">More Info</a>
                        </div>
                    </div>
                </div>
                """,
                unsafe_allow_html=True
            )
    else:
        st.error("🚫 Oops! We couldn't find that title. Try something like 'Stranger Things' or 'The Witcher'")

# Footer
st.markdown("---")
st.markdown("""
    <div style="text-align: center; padding: 1rem; color: #888;">
        <p>Made with ❤️ using Streamlit | Data from Netflix</p>
        <p>🎥 Keep Watching, Keep Discovering 🎥</p>
    </div>
""", unsafe_allow_html=True)

Writing app.py


In [ ]:
from pyngrok import ngrok
import os

ngrok.set_auth_token("")

# Kill any previous tunnels
ngrok.kill()

# Running streamlit app in background
os.system("streamlit run app.py &")

# Creating a public tunnel
public_url = ngrok.connect(8501, "http")
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://254b8e72bd20.ngrok-free.app" -> "http://localhost:8501"
